In [4]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [5]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        # features 1, hidden size 51
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for input_t in input.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [27]:
i = 0
# for x in np.arange(5):
#     print('i: ' + str(i))
#     i += 1
for input_t in input.split(1, dim=1):
    if i == 1:
        print(input_t.shape)
    # print('i: ' + str(i))
    i += 1

torch.Size([97, 1])


In [6]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--steps', type=int, default=15, help='steps to run')
# opt = parser.parse_args()
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)
# load data and make training set
data = torch.load('traindata.pt')
input = torch.from_numpy(data[3:, :-1])
target = torch.from_numpy(data[3:, 1:])
test_input = torch.from_numpy(data[:3, :-1])
test_target = torch.from_numpy(data[:3, 1:])
# build the model
seq = Sequence()
seq.double()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)

In [13]:
#begin to train
# for i in range(opt.steps):
i = 0
print('STEP: ', i)
def closure():
    optimizer.zero_grad()
    out = seq(input)
    loss = criterion(out, target)
    print('loss:', loss.item())
    loss.backward()
    return loss
optimizer.step(closure)

STEP:  0
loss: 0.0012797646167832567
loss: 0.001169055495471881
loss: 0.0011498916013729771
loss: 0.0011288245291281454
loss: 0.0010630561341020393
loss: 0.0009564006443272563
loss: 0.00082108275899727
loss: 0.0007670796029156168
loss: 0.0007294716423584051
loss: 0.000724655816981735
loss: 0.0007206211393655977
loss: 0.0007126686053115696
loss: 0.0006961310498781834
loss: 0.000664110181155962
loss: 0.000610121070230019
loss: 0.0005285228387129998
loss: 0.00041272507103123365
loss: 0.000330215794523346
loss: 0.0003121712149390207
loss: 0.0003235219220500932


tensor(0.0013, dtype=torch.float64, grad_fn=<MseLossBackward0>)

In [12]:
# begin to predict, no need to track gradient here
with torch.no_grad():
    future = 1000
    pred = seq(test_input, future=future)
    loss = criterion(pred[:, :-future], test_target)
    print('test loss:', loss.item())
    y = pred.detach().numpy()
# draw the result
plt.figure(figsize=(30,10))
plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
def draw(yi, color):
    plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
    plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
draw(y[0], 'r')
draw(y[1], 'g')
draw(y[2], 'b')
plt.savefig('predict%d.pdf'%i)
plt.close()

STEP:  0
loss: 0.5023738122475572
loss: 0.49856639379435636
loss: 0.47901196061152884
loss: 0.4463349021484236
loss: 0.35406310257492596
loss: 0.20507016617681337
loss: 1.396053156117307
loss: 0.03249441148472198
loss: 0.02993487583960496
loss: 0.028326821011534428
loss: 0.026830612218824056
loss: 0.023771201989991487
loss: 0.018901413504548843
loss: 0.010646818233208458
loss: 0.008725752090268811
loss: 0.007872181287777372
loss: 0.00547784274959501
loss: 0.004051933564064951
loss: 0.0027296227011610404
loss: 0.001540265276982263
test loss: 0.0013000876156961515
